In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import math
from math import acos, degrees
from scipy.signal import find_peaks
import os.path
import glob
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import entropy
import pylab as pl
from numpy.fft import fft
from scipy import stats
import numpy
from scipy import signal



In [2]:
chunk_width = 250
#centr_rang = 110

In [3]:
#Function to extract angle using 3 points coordinate
def angle3pt(a, b, c):
#    """Counterclockwise angle in degrees by turning from c to a around b
#        Returns a float between 0.0 and 360.0"""
    ang = math.degrees(
    math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    return ang + 360 if ang < 0 else ang


In [4]:
def getridofAngleJumps(alpha):
    alpha_rad = [x*(np.pi)/180 for x in alpha]
    alpha_rad = np.array(alpha_rad)
    alpha_rad[~np.isnan(alpha_rad)] = np.unwrap(alpha_rad[~np.isnan(alpha_rad)])
    alpha_unwrap= np.degrees(alpha_rad)
    return alpha_unwrap

In [5]:
##function to get rid of ugly jumps due to angle range [0-360], input alpha that is a list of angles
#def getridofAngleJumps(x):
#    n = [ 0, *np.diff(x) ]
##    #get rid of negative jumps
#    for i in range(len(n)):
#        if n[i] < -300:
#            while x[i] < 200: #or x[i] == 'NaN':
#                x[i] = x[i]+360
#                i += 1
#                if x[i] == 'NaN':
#                    i += 1
#    #get rid also of positive jumps
##        if n[i] > 300:
##            while x[i] > 300:
##                x[i] = x[i]-360
##                i += 1        
#    return x


In [6]:
def smooth_acausal(x,frequency = 0.300):
    #b, a = signal.butter(8, 0.150)
    sos = signal.butter(4, frequency, output='sos')
    y = signal.sosfiltfilt(sos, x)
    return y

In [7]:
def smooth(x,window_len=20,window='hanning'):
#    """smooth the data using a window with requested size.
#    
#    This method is based on the convolution of a scaled window with the signal.
#    The signal is prepared by introducing reflected copies of the signal 
#    (with the window size) in both ends so that transient parts are minimized
#    in the begining and end part of the output signal.
#    
#    input:
#        x: the input signal 
#        window_len: the dimension of the smoothing window; should be an odd integer
#        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
#            flat window will produce a moving average smoothing.
#
#    output:
#        the smoothed signal
#        
#    example:
#
#    t=linspace(-2,2,0.1)
#    y=smooth(x)
#    
#    
#    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
#    scipy.signal.lfilter
 
#    TODO: the window parameter could be the window itself if an array instead of a string
#    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.

    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y


In [8]:
#Function to extract RidgeX trajectory from excel file
def RidgeX_excel_to_array_preprocessed(file_path, chunk_width, trial_no):
    #Read csv file tail markers
    RidgeX = pd.read_csv(file_path[trial_no])

    #take just numeric values
    RidgeX=pd.to_numeric(RidgeX.iloc[:,0])

 
    return smooth(RidgeX.values)


In [9]:
#Function to plot tail angle trajectory
def plot_TailAngle(file_path, chunk_width, i, c):
    #Read csv file tail markers
    df = pd.read_csv(file_path[i])
#    df = pd.read_csv(file_path)
    #Rename marker columns
    df.rename(columns={'DLC_resnet50_Ridge_walkMay27shuffle1_1000000':'tail1_x',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.1':'tail1_y',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.2':'tail1_lik',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.3':'tail2_x',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.4':'tail2_y',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.5':'tail2_lik',                       
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.6':'tail3_x',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.7':'tail3_y',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.8':'tail3_lik'}, 
                 inplace=True)

    #take just numeric values
    df.tail1_x=pd.to_numeric(df.tail1_x[c-chunk_width:c+chunk_width])
    df.tail1_y=pd.to_numeric(df.tail1_y[c-chunk_width:c+chunk_width])
    df.tail2_x=pd.to_numeric(df.tail2_x[c-chunk_width:c+chunk_width])
    df.tail2_y=pd.to_numeric(df.tail2_y[c-chunk_width:c+chunk_width])
    df.tail3_x=pd.to_numeric(df.tail3_x[c-chunk_width:c+chunk_width])
    df.tail3_y=pd.to_numeric(df.tail3_y[c-chunk_width:c+chunk_width])
    df.tail1_lik=pd.to_numeric(df.tail1_lik[c-chunk_width:c+chunk_width])
    df.tail2_lik=pd.to_numeric(df.tail2_lik[c-chunk_width:c+chunk_width])
    df.tail3_lik=pd.to_numeric(df.tail3_lik[c-chunk_width:c+chunk_width])
    
    #substitute low likelihood points with NaN
    #df.tail1_x[]=np.nan
    lik_thresh = 0.05
    df.tail1_x.where((df.tail1_lik>lik_thresh),np.NaN,inplace=True)
    df.tail1_y.where((df.tail1_lik>lik_thresh),np.NaN,inplace=True)
    df.tail2_x.where((df.tail2_lik>lik_thresh),np.NaN,inplace=True)
    df.tail2_y.where((df.tail2_lik>lik_thresh),np.NaN,inplace=True)
    df.tail3_x.where((df.tail3_lik>lik_thresh),np.NaN,inplace=True)
    df.tail3_y.where((df.tail3_lik>lik_thresh),np.NaN,inplace=True)




    #Compute and plot tail angle in panda dataframe as last column
    angles=[]
    for i in range(df.shape[0]):
        #x1,y1=df.tail1_x[i],df.tail1_y[i]
        vertical = np.array([df.tail1_x[i],df.tail1_y[i]+10])
        tail1 = np.array([df.tail1_x[i],df.tail1_y[i]])
        tail2 = np.array([df.tail2_x[i],df.tail2_y[i]])    
        tail3 = np.array([df.tail3_x[i],df.tail3_y[i]])
    

    #Change below to decide 3 points to determine angle
        angle = angle3pt(tail2, tail1, vertical)
        #Append
        angles.append(round(angle,2))
    df['Angles']=angles
    df.head()
    
    #Create a list comprehension by chunking x (centroid X trajectory) in 'chunk_width' frames and alpha (tail angle traj)
    #x = CentroidXY.X
    alpha = df.Angles
    #Get alpha value at perturbation time to centre the trace to that value
#    alpha_centred = alpha[tot_peaks]
    #Apply function to get rid of angle jumps
    alpha = getridofAngleJumps(alpha)
    #Apply function to smooth
    alpha = smooth(alpha)
#    TailAngle_traj = alpha[tot_peaks-chunk_width:tot_peaks+chunk_width]-[alpha[tot_peaks]-alpha_centred]# for i in tot_peaks]
#!    return [TailAngle_traj, alpha]
    return alpha

In [10]:
#Function to plot Right Paw angle trajectory
def plot_RPAngle(file_path, chunk_width, trial_no):
    #Read csv file tail markers
    df = pd.read_csv(file_path[trial_no])
#    df = pd.read_csv(file_path)

    #Rename marker columns
    df.rename(columns={'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.9':'LP_x',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.10':'LP_y',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.11':'LP_lik',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.24':'RA_x',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.25':'RA_y',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.26':'RA_lik',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.21':'RP_x',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.22':'RP_y',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.23':'RP_lik'}, 
                 inplace=True)

    #take just numeric values
    df.RA_x=pd.to_numeric(df.RA_x[2:])
    df.RA_y=pd.to_numeric(df.RA_y[2:])
    df.RP_x=pd.to_numeric(df.RP_x[2:])
    df.RP_y=pd.to_numeric(df.RP_y[2:])
    df.LP_x=pd.to_numeric(df.LP_x[2:])
    df.LP_y=pd.to_numeric(df.LP_y[2:])
    df.RA_lik=pd.to_numeric(df.RA_lik[2:])
    df.RP_lik=pd.to_numeric(df.RP_lik[2:])
    df.LP_lik=pd.to_numeric(df.LP_lik[2:])

    
    #substitute low likelihood points with NaN
    #df.tail1_x[]=np.nan
    lik_thresh = 0.1
    df.RA_x.where((df.RA_lik>lik_thresh),np.NaN,inplace=True)
    df.RA_y.where((df.RA_lik>lik_thresh),np.NaN,inplace=True)
    df.RP_x.where((df.RP_lik>lik_thresh),np.NaN,inplace=True)
    df.RP_y.where((df.RP_lik>lik_thresh),np.NaN,inplace=True)
    df.LP_x.where((df.LP_lik>lik_thresh),np.NaN,inplace=True)
    df.LP_y.where((df.LP_lik>lik_thresh),np.NaN,inplace=True)
    
    #extract RP x and LP x
    RP_x =  df.RP_x
    LP_x =  df.LP_x




    #Compute and plot tail angle in panda dataframe as last column
    angles=[]
    for i in range(df.shape[0]):
        vertical = np.array([df.RA_x[i],df.RA_y[i]+10])
        RA = np.array([df.RA_x[i],df.RA_y[i]])
        RP = np.array([df.RP_x[i],df.RP_y[i]])

        angle = angle3pt(RP, RA, vertical)
        #Append
        angles.append(round(angle,2))
    df['Angles']=angles
    df.head()
    
    #Create a list comprehension by chunking x (centroid X trajectory) in 'chunk_width' frames and alpha (tail angle traj)
    #x = CentroidXY.X
    alpha = df.Angles
    #Get alpha value at perturbation time to centre the trace to that value
#    alpha_centred = alpha[tot_peaks]
    #Apply function to get rid of angle jumps
    alpha = getridofAngleJumps(alpha)
    #Apply function to smooth
    alpha = smooth(alpha)
#    TailAngle_traj = alpha[tot_peaks-chunk_width:tot_peaks+chunk_width]-[alpha[tot_peaks]-alpha_centred]# for i in tot_peaks]
#    RP_x = RP_x[tot_peaks-chunk_width:tot_peaks+chunk_width]-[RP_x[tot_peaks]]
#    LP_x = LP_x[tot_peaks-chunk_width:tot_peaks+chunk_width]-[LP_x[tot_peaks]]

#!    return [TailAngle_traj, alpha]
    return [alpha, smooth(RP_x), smooth(LP_x)]

In [11]:
#Function to plot Right Paw angle trajectory
def plot_LPAngle(file_path, chunk_width, trial_no):
    #Read csv file tail markers
    df = pd.read_csv(file_path[trial_no])
#    df = pd.read_csv(file_path)

    #Rename marker columns
    df.rename(columns={'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.9':'LP_x',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.10':'LP_y',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.11':'LP_lik',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.12':'LA_x',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.13':'LA_y',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.14':'LA_lik',
                       
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.24':'RA_x',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.25':'RA_y',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.26':'RA_lik',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.21':'RP_x',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.22':'RP_y',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.23':'RP_lik'}, 
                 inplace=True)

    #take just numeric values
    df.RA_x=pd.to_numeric(df.RA_x[2:])
    df.LA_x=pd.to_numeric(df.LA_x[2:])

    df.RA_y=pd.to_numeric(df.RA_y[2:])
    df.LA_y=pd.to_numeric(df.LA_y[2:])

    df.RP_x=pd.to_numeric(df.RP_x[2:])
    df.RP_y=pd.to_numeric(df.RP_y[2:])
    df.LP_x=pd.to_numeric(df.LP_x[2:])
    df.LP_y=pd.to_numeric(df.LP_y[2:])
    
    df.RA_lik=pd.to_numeric(df.RA_lik[2:])
    df.LA_lik=pd.to_numeric(df.LA_lik[2:])
    df.RP_lik=pd.to_numeric(df.RP_lik[2:])
    df.LP_lik=pd.to_numeric(df.LP_lik[2:])

    
    #substitute low likelihood points with NaN
    #df.tail1_x[]=np.nan
    lik_thresh = 0.1
    df.RA_x.where((df.RA_lik>lik_thresh),np.NaN,inplace=True)
    df.LA_x.where((df.LA_lik>lik_thresh),np.NaN,inplace=True)

    df.RA_y.where((df.RA_lik>lik_thresh),np.NaN,inplace=True)
    df.LA_y.where((df.LA_lik>lik_thresh),np.NaN,inplace=True)

    df.RP_x.where((df.RP_lik>lik_thresh),np.NaN,inplace=True)
    df.RP_y.where((df.RP_lik>lik_thresh),np.NaN,inplace=True)
    df.LP_x.where((df.LP_lik>lik_thresh),np.NaN,inplace=True)
    df.LP_y.where((df.LP_lik>lik_thresh),np.NaN,inplace=True)
    
    #extract RP x and LP x
    RP_x =  df.RP_x
    LP_x =  df.LP_x




    #Compute and plot tail angle in panda dataframe as last column
    angles=[]
    for i in range(df.shape[0]):
        vertical = np.array([df.LA_x[i],df.LA_y[i]+10])
        LA = np.array([df.LA_x[i],df.LA_y[i]])
        LP = np.array([df.LP_x[i],df.LP_y[i]])

        angle = angle3pt(LP, LA, vertical)
        #Append
        angles.append(round(angle,2))
    df['Angles']=angles
    df.head()
    
    #Create a list comprehension by chunking x (centroid X trajectory) in 'chunk_width' frames and alpha (tail angle traj)
    #x = CentroidXY.X
    alpha = df.Angles
    #Get alpha value at perturbation time to centre the trace to that value
#    alpha_centred = alpha[tot_peaks]
    #Apply function to get rid of angle jumps
    alpha = getridofAngleJumps(alpha)
    #Apply function to smooth
    alpha = smooth(alpha)
#    TailAngle_traj = alpha[tot_peaks-chunk_width:tot_peaks+chunk_width]-[alpha[tot_peaks]-alpha_centred]# for i in tot_peaks]
#    RP_x = RP_x[tot_peaks-chunk_width:tot_peaks+chunk_width]-[RP_x[tot_peaks]]
#    LP_x = LP_x[tot_peaks-chunk_width:tot_peaks+chunk_width]-[LP_x[tot_peaks]]

#!    return [TailAngle_traj, alpha]
    return [alpha, smooth(RP_x), smooth(LP_x)]

In [12]:
#Function to plot Right Paw angle trajectory
def plot_HipAngle(file_path, chunk_width, trial_no):
    #Read csv file tail markers
    df = pd.read_csv(file_path[trial_no])
#    df = pd.read_csv(file_path)

    #Rename marker columns
    df.rename(columns={'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.15':'LH_x',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.16':'LH_y',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.17':'LH_lik',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.18':'RH_x',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.19':'RH_y',
                          'DLC_resnet50_Ridge_walkMay27shuffle1_1000000.20':'RH_lik'}, 
                 inplace=True)

    #take just numeric values
    df.LH_x=pd.to_numeric(df.LH_x[2:])
    df.LH_y=pd.to_numeric(df.LH_y[2:])
    df.RH_x=pd.to_numeric(df.RH_x[2:])
    df.RH_y=pd.to_numeric(df.RH_y[2:])
    df.RH_lik=pd.to_numeric(df.RH_lik[2:])
    df.LH_lik=pd.to_numeric(df.LH_lik[2:])

    
    #substitute low likelihood points with NaN
    #df.tail1_x[]=np.nan
    lik_thresh = 0.1
    df.LH_x.where((df.LH_lik>lik_thresh),np.NaN,inplace=True)
    df.LH_y.where((df.LH_lik>lik_thresh),np.NaN,inplace=True)
    df.RH_x.where((df.RH_lik>lik_thresh),np.NaN,inplace=True)
    df.RH_y.where((df.RH_lik>lik_thresh),np.NaN,inplace=True)

    
    #extract RP x and LP x
    LH_x =  df.LH_x
    LH_y =  df.LH_y
    RH_x =  df.RH_x    
    RH_y =  df.RH_y




    #Compute and plot tail angle in panda dataframe as last column
    angles=[]
    for i in range(df.shape[0]):
        vertical = np.array([df.RH_x[i],df.RH_y[i]+10])
        RH = np.array([df.RH_x[i],df.RH_y[i]])
        LH = np.array([df.LH_x[i],df.LH_y[i]])

        angle = angle3pt(LH, RH, vertical)
        #Append
        angles.append(round(angle,2))
    df['Angles']=angles
    df.head()
    
    #Create a list comprehension by chunking x (centroid X trajectory) in 'chunk_width' frames and alpha (tail angle traj)
    #x = CentroidXY.X
    alpha = df.Angles
    #Get alpha value at perturbation time to centre the trace to that value
#    alpha_centred = alpha[tot_peaks]
    #Apply function to get rid of angle jumps
    alpha = getridofAngleJumps(alpha)
    #Apply function to smooth
    alpha = smooth(alpha)
#    TailAngle_traj = alpha[tot_peaks-chunk_width:tot_peaks+chunk_width]-[alpha[tot_peaks]-alpha_centred]# for i in tot_peaks]
#    RP_x = RP_x[tot_peaks-chunk_width:tot_peaks+chunk_width]-[RP_x[tot_peaks]]
#    LP_x = LP_x[tot_peaks-chunk_width:tot_peaks+chunk_width]-[LP_x[tot_peaks]]

#!    return [TailAngle_traj, alpha]
    return alpha

In [13]:
#Function to extract centroid X Y trajectory
def extract_Centroid(file_path, chunk_width, trial_no):
    #Read csv file tail markers
    CentroidXY = pd.read_csv(file_path[trial_no])

    CentroidXY.rename(columns={'NaN':'X',
                              'NaN.1':'Y'}, 
                     inplace=True)
    #take just numeric values
    CentroidXY.Centroid_x=pd.to_numeric(CentroidXY.X)
    CentroidXY.Centroid_y=pd.to_numeric(CentroidXY.Y)

    #extract Centroid x and y
    CentroidX =  np.array(CentroidXY.Centroid_x)
    CentroidY =  np.array(CentroidXY.Centroid_y)
    return CentroidX, CentroidY


In [14]:
def find_delay(a, b):
    corr_a_b = np.correlate(a-np.mean(a), b-np.mean(b), mode = 'full')
    delay = np.where(corr_a_b == numpy.amin(corr_a_b))# -(np.size(corr_a_b)+1)/2
    return delay[0]-(np.size(corr_a_b)+1)/2

In [15]:
#Function to plot Right Paw angle trajectory
def plot_Centroid_edge_dist(file_path, chunk_width, i):
    #Read csv file tail markers
    df = pd.read_csv(file_path[i])

##Open Centroid file from top camera
#CentroidXY = pd.read_csv('C:/Users/Salvo/Desktop/Ridge/DLC_videos/Videos_to_analyzeDLC/Ridge_MiceS20-S24_16thApril/perturbation_8mm_1/8_S22/Centroid.csv')

    #Select 1st column csv file
    matrix2 = df[df.columns[0]]#.as_matrix()
    Centroid1stcol = matrix2.tolist() #file 1st column


#    CentroidXY.rename(columns={'NaN':'dist'}, 
#                     inplace=True)
    #take just numeric values
    Centroid1stcol = np.array(pd.to_numeric(Centroid1stcol))
#    CentroidXY.Centroid_y=pd.to_numeric(CentroidXY.Y)

    #extract Centroid x and y
#    CentroidX =  CentroidXY.Centroid_x
#    CentroidY =  CentroidXY.Centroid_y
    
#    Centroid_list = CentroidX[tot_peaks-chunk_width:tot_peaks+centr_rang-chunk_width]#-[CentroidX[tot_peaks]]
    
#    Centroid_list = CentroidX[tot_peaks-chunk_width-100:tot_peaks-100]-[CentroidX[tot_peaks-100]]
    return smooth(Centroid1stcol)

In [16]:
#np.arange(len(file_to_open)-25)


In [17]:

def firstNonNan(listfloats):
    i = 0
    for item in listfloats:
        i += 1
        if math.isnan(item) == False:
            return i

#firstNonNan(t)

In [18]:
def HIST_MI_2_var(a, b):
    fig = plt.figure(figsize=(10,14))
    ax1 = plt.subplot(311)
    ax2 = plt.subplot(312)
    hist_centr = ax1.hist(a, density=True, bins=30, color = 'orange')  # `density=False` would make counts
    hist_tail = ax2.hist(b, density=True, bins=30, color = 'blue')  # `density=False` would make counts
#    ent_cent = entropy(hist_centr[0], base=2)
#    ent_tail = entropy(hist_tail[0], base=2)
    MI_cent_tail = metrics.mutual_info_score(hist_centr[0], hist_tail[0])
    return MI_cent_tail

In [19]:
def get_delay_array(var1, var2):
    var1 = var1[~np.isnan(var1)] #centroid
    var2 = var2[~np.isnan(var2)] #TA
    #take the mean out
#    var1 = var1-np.mean(var1)
#    var2 = var2-np.mean(var2)

    corr_a_b = np.correlate(var1, var2, mode = 'full')
    #norm_corr_a_b = np.correlate(var2/np.std(var2), var1/np.std(var1), mode = 'full')
    cc_trace_midpoint = len(corr_a_b)
    delay = np.argmax(abs(corr_a_b))-(cc_trace_midpoint/2)+1 #Get the delay of the absolute max peak
    max_peak = max(corr_a_b)#, key=abs)
#    max_peak = abs(max(corr_a_b, key=abs))
    return delay, max_peak

In [20]:
def get_fft(chunk_width, data):
    # Number of sample points
    N = chunk_width*2
    # sample spacing
    T = 1/300
    x = np.linspace(0.0, N*T, N)
    y = data
    yf = fft(y)
    xf = np.linspace(0, 1/(2*T), N//2)
#    plt.plot(xf, 2/N * np.abs(yf[0:N//2]))
#    plt.grid()
#    plt.show()
    return xf, yf

In [21]:
def findpeaks_extractchunk(x_diff, x, y, z, w, w_x, t, other_step_x, ba, threshold_height, chunk_width_step):
    peaks, _ = find_peaks(x_diff, height=threshold_height, distance = 50, prominence = 1)
    out_step = []
    out_TA = []
    out_HA = []
    out_cent = []
    out_RstepAng = []
    out_cent_X = []
    out_ba = []
    out_contra_step_x = []
    for i in np.arange(len(peaks)):
        chunk_trial_step = x[peaks[i]-chunk_width_step:peaks[i]+chunk_width_step]
        chunk_trial_contra_step = other_step_x[peaks[i]-chunk_width_step:peaks[i]+chunk_width_step]
        chunk_trial_TA = y[peaks[i]-chunk_width_step:peaks[i]+chunk_width_step]
        chunk_trial_HA = z[peaks[i]-chunk_width_step:peaks[i]+chunk_width_step]
        chunk_trial_cent = w[peaks[i]-chunk_width_step:peaks[i]+chunk_width_step]
        chunk_trial_centX = w_x[peaks[i]-chunk_width_step:peaks[i]+chunk_width_step]
        chunk_trial_RstepAng = t[peaks[i]-chunk_width_step:peaks[i]+chunk_width_step]
        chunk_trial_BA = ba[peaks[i]-chunk_width_step:peaks[i]+chunk_width_step]
        

        out_step.append(chunk_trial_step)
        out_TA.append(chunk_trial_TA)
        out_HA.append(chunk_trial_HA)
        out_cent.append(chunk_trial_cent)
        out_cent_X.append(chunk_trial_centX)
        out_ba.append(chunk_trial_BA)
        out_contra_step_x.append(chunk_trial_contra_step)
        #transpose all traces of step angle greater than 360 back to 0
        if np.nanmean(chunk_trial_RstepAng) > 250:
            out_RstepAng.append(chunk_trial_RstepAng-360)
        elif np.nanmean(chunk_trial_RstepAng) < -150:
            out_RstepAng.append(chunk_trial_RstepAng+360)   
        else:
            out_RstepAng.append(chunk_trial_RstepAng)


    
    return out_step, out_TA, out_HA, out_cent, out_cent_X, out_RstepAng, out_contra_step_x, out_ba

In [22]:
import matplotlib.pyplot as plt
import matplotlib.text as mpl_text

class AnyObject(object):
    def __init__(self, text, color):
        self.my_text = text
        self.my_color = color

class AnyObjectHandler(object):
    def legend_artist(self, legend, orig_handle, fontsize, handlebox):
        print(orig_handle)
        x0, y0 = handlebox.xdescent, handlebox.ydescent
        width, height = handlebox.width, handlebox.height
        patch = mpl_text.Text(x=0, y=0, text=orig_handle.my_text, color=orig_handle.my_color, verticalalignment=u'baseline', 
                                horizontalalignment=u'left', multialignment=None, 
                                fontproperties=None, rotation=45, linespacing=None, 
                                rotation_mode=None)
        handlebox.add_artist(patch)
        return patch

In [23]:
def delete_bad_traces_FromList(Traces_List, idx_to_eliminate):
    for l in np.arange(len(idx_to_eliminate)):
        for i in np.arange(len(Traces_List)):
            for j in np.arange(len(Traces_List[i])):
                if len(Traces_List[i][j]) == 200:
                    if i == idx_to_eliminate[l][0] and j == idx_to_eliminate[l][1]:
                        Traces_List[i][j] = [] 
    return Traces_List

In [24]:
def check_trace(trace):
    if np.nanmean(trace) <-50:
        trace = trace + 360
#    if np.nanmean(trace) >400:
#        trace = trace - 360
    return trace

In [25]:
def check_trace_within_0_to_150(trace):
    if np.nanmean(trace) <-50:
        trace = trace + 360
        if np.nanmean(trace) <-50:
            trace = trace + 360
#    elif np.nanmean(trace[0:60]) >150:
#        trace = []
    else:
        trace = trace
    return trace

In [26]:
def fill_nan(arr):
#     mask = np.isnan(arr)
#     idx = np.where(~mask,np.arange(mask.size),0)
#     np.maximum.accumulate(idx, out=idx)
#     arr[mask] = arr[idx]
    df = pd.DataFrame(data=arr.flatten())
    df = df.fillna(value=None, method='backfill', axis=None, limit=70, downcast=None)
    arr = df.values
#    print(type(arr))
    return arr.ravel()

In [30]:
### Organize all data into python dict
from collections import defaultdict

search_key_path = ['*4mm*', '*5mm*', '*8mm*', '*10mm*']
search_key = ['4mm', '5mm', '8mm', '10mm']

dict_ridge_all = defaultdict(dict)
for j in np.arange(len(search_key)):
    data_location = "C://Users//SWAS92//Desktop//side_cam//"
    RidgeX_ExcelList_to_open = glob.glob(os.path.join(os.path.sep, data_location, search_key_path[j], 'Ridge_X//*.csv'))
    TA_ExcelList_to_open = glob.glob(os.path.join(os.path.sep, data_location, search_key_path[j], 'side_cam//*//*.csv'))
    Centroid_ExcelList_to_open = glob.glob(os.path.join(os.path.sep, data_location, search_key_path[j], 'Centroid_XY//*.csv'))
    BodyAxis_ExcelList_to_open = glob.glob(os.path.join(os.path.sep, data_location, search_key_path[j], 'BodyAxis//*.csv'))

#    print(RidgeX_ExcelList_to_open)
#    dict_ridge = {}
    for i in np.arange(len(RidgeX_ExcelList_to_open)): # len(peaks)
        #fig, ax1 = plt.subplots(1, 1, figsize=(15,6))
        #Extract arrays
        RidgeX_traj = RidgeX_excel_to_array_preprocessed(RidgeX_ExcelList_to_open, chunk_width, i)
        CentroidX_traj, CentroidY_traj = extract_Centroid(Centroid_ExcelList_to_open, chunk_width, i)
        BodyAxis_traj = RidgeX_excel_to_array_preprocessed(BodyAxis_ExcelList_to_open, chunk_width, i)
        #Extract traces of Centroid and Tail Angle around the time frame when the mouse is at the ridge center
        a = firstNonNan(CentroidX_traj)
        b = round((np.size(CentroidX_traj) - np.count_nonzero(np.isnan(CentroidX_traj)))/2)
        c = a + b

        #Take tail angle traj after extracting chunk of traj of interest around c
        TailAngle_traj = plot_TailAngle(TA_ExcelList_to_open, chunk_width, i, c)

        RidgeX_traj_chunk = fill_nan(RidgeX_traj[c-chunk_width:c+chunk_width])
        TailAngle_traj_chunk = fill_nan(TailAngle_traj[c-chunk_width:c+chunk_width])
        #TailAngle_traj_chunk_corrected = check_trace(TailAngle_traj_chunk)
        CentroidX_traj_chunk = (CentroidX_traj[c-chunk_width:c+chunk_width])
        CentroidY_traj_chunk = (CentroidY_traj[c-chunk_width:c+chunk_width])
        BodyAxis_traj_chunk = fill_nan(BodyAxis_traj[c-chunk_width:c+chunk_width])
        HipAngle_traj = plot_HipAngle(TA_ExcelList_to_open, chunk_width, i)
        HipAngle_traj_chunk = fill_nan(HipAngle_traj[c-chunk_width:c+chunk_width])

#         CentroidX_chunk  = CentroidX_traj[c-chunk_width:c+chunk_width]
#         CentroidX_chunk_withoutNaN = CentroidX_traj[~np.isnan(CentroidX_traj)] #drop NaN
     
        #Compute R and L step 
        [RPAngle_traj, RP_x, LP_x]   = plot_RPAngle(TA_ExcelList_to_open, chunk_width, i)
        RP_x = RP_x[c-chunk_width:c+chunk_width]
        LP_x = LP_x[c-chunk_width:c+chunk_width]   
        RPAngle_traj_chunk = fill_nan(RPAngle_traj[c-chunk_width:c+chunk_width])
        
        #divide trial into step-based chunks RIGHT
        x_r = RP_x-np.nanmean(RP_x)
        x_l = LP_x-np.nanmean(LP_x)
        x_r_diff = np.diff(RP_x-np.nanmean(RP_x))
        x_l_diff = np.diff(-(LP_x-np.nanmean(-LP_x)))
        y_r = TailAngle_traj_chunk#-np.nanmean(TailAngle_traj)
        y_l = TailAngle_traj_chunk#-np.nanmean(TailAngle_traj)
        w = CentroidY_traj_chunk#-np.nanmean(Centroid_DIST_traj)#*50
        z = HipAngle_traj_chunk#-np.nanmean(HipAngle_traj)
        t = RPAngle_traj_chunk#-np.nanmean(RPAngle_traj)
        ba = BodyAxis_traj_chunk
        w_x = CentroidX_traj_chunk
        step_chunk_R, TA_chunk_R, HA_chunk_R, cent_chunk_R, cent_chunk_XR, StepAngle_chunk_R, step_chunk_L_wrt_Rstep, \
        ba_wrt_Rstep = findpeaks_extractchunk(x_r_diff, x_r, y_r, z, w, w_x, t, x_l, ba, 0.9, chunk_width//2)
        
        #divide trial into step-based chunks LEFT
        step_chunk_L, TA_chunk_L, HA_chunk_L, cent_chunk_L, cent_chunk_XL, StepAngle_chunk_L, step_chunk_R_wrt_Lstep, \
        ba_wrt_Lstep = findpeaks_extractchunk(x_l_diff, x_l, y_l, z, w, w_x, t, x_r, ba, 0.9, chunk_width//2)
        
        
        #Decide here what variables to plot in the three figures
        var1 = np.array(RidgeX_traj_chunk)
        var2 = np.array(TailAngle_traj_chunk)
        var3 = np.array(CentroidX_traj_chunk)
        var4 = step_chunk_R
        var5 = TA_chunk_R
        var6 = step_chunk_L
        var7 = TA_chunk_L
        var8 = StepAngle_chunk_R
        var9 = StepAngle_chunk_L
        var10 = cent_chunk_R
        var11 = cent_chunk_L
        var12 = cent_chunk_XR
        #var13 = cent_DistChunk_R
        var13 = HA_chunk_R
        var14 = StepAngle_chunk_R
        var15 = step_chunk_L_wrt_Rstep
        var16 = ba_wrt_Rstep
        var17 = ba_wrt_Lstep
        var18 = HA_chunk_L
            
        #Make dict
        key_file_name = os.path.basename(RidgeX_ExcelList_to_open[i])
        dict_ridge_all[search_key[j]][key_file_name] = [var1, var2, var3, var4, var5, var6, var7, \
                                                        var8, var9, var10, var11, var12, var13, var14, var15, \
                                                        var16, var17, var18]
   

C:\Program Files\Anaconda3\envs\DLC-GPU\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # Remove the CWD from sys.path while we load stuff.
C:\Program Files\Anaconda3\envs\DLC-GPU\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # This is added back by InteractiveShellApp.init_path()
C:\Program Files\Anaconda3\envs\DLC-GPU\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: Mean of empty slice
C:\Program Files\Anaconda3\envs\DLC-GPU\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: Mean of empty slice


In [ ]:
def assign_dict_value_ridge_pos(dict_ridge):
    #Divide trials based on ridge position. Assign -1 for left tilt, +1 for right and 0 for no tilts. Append to 4th col
    #Changed the threshold from 5000 to 10000 bcs M53 detected many no pert trials as pert
    key_list = list(dict_ridge.keys())

    for i in np.arange(len(key_list)):

        ridge_array = dict_ridge[key_list[i]][0]
        ridge_array_translated_nonNaN = ridge_array[~np.isnan(ridge_array)]
        ridge_array_translated_nonNaN_mean_centered = ridge_array_translated_nonNaN-  \
        np.nanmean(smooth(ridge_array_translated_nonNaN[20:40]))
        ridge_array_translated_int = np.trapz(smooth(ridge_array_translated_nonNaN_mean_centered, 50))
        ridge_array_translated_nonNaN_mean_centered_diff_max = max(np.diff(ridge_array_translated_nonNaN_mean_centered))
        if ridge_array_translated_int < -3000 and ridge_array_translated_nonNaN_mean_centered_diff_max<40:
            dict_ridge[key_list[i]].append(-1)
            #print(ridge_array_translated_nonNaN_mean_centered_diff_max)
        elif ridge_array_translated_int > +3000 and ridge_array_translated_nonNaN_mean_centered_diff_max<40:
            dict_ridge[key_list[i]].append(1)
    #        print(ridge_array_translated_int)

        else:
            dict_ridge[key_list[i]].append(0)    
    
    return dict_ridge

In [ ]:
def eliminate_pert_trials_from_dict(dict_ridge):
    values_list = list(dict_ridge.values())
    key_list = list(dict_ridge.keys())
    for i in np.arange(len(key_list)):
        Ridge_classvalue = values_list[i][-1]
        if Ridge_classvalue == 1 or Ridge_classvalue == -1:
            key_to_be_deleted = key_list[i]
            dict_ridge.pop(key_to_be_deleted, None)    
    return dict_ridge

In [ ]:
def excludeNaNTATraces(dict_ridge):
    #Exclude from dict all trials where the TA traj is mostly NaN
    values_list = list(dict_ridge.values())
    key_list = list(dict_ridge.keys())

    for i in np.arange(len(key_list)):
        TA_traj = values_list[i][1]
        no_of_nan_TAtraj = list(np.isnan(TA_traj))
        count_NaN = no_of_nan_TAtraj.count(1)
        if count_NaN>70:
            key_to_be_deleted = key_list[i]
            dict_ridge.pop(key_to_be_deleted, None)
        elif len(TA_traj) == 0:
            key_to_be_deleted = key_list[i]
            dict_ridge.pop(key_to_be_deleted, None)    
    return dict_ridge      

In [ ]:
def transpose_traces_0_360_range(dict_ridge):
    #Transpose from dict all trials where the TA traj is outsude [0, 360] and append to 13th column 
    values_list = list(dict_ridge.values())
    key_list = list(dict_ridge.keys())
    #dict_TA_transpose = {}
    for i in np.arange(len(key_list)):
        TA_traj = values_list[i][4]
        key_to_change = key_list[i]
        TA_to_append = []
        for j in np.arange(len(TA_traj)):
            if np.nanmean(TA_traj[j])>400:
                TA_traj_pushed_down = TA_traj[j]-360
                TA_to_append.append(TA_traj_pushed_down)      
            elif np.nanmean(TA_traj[j])<-100:
                TA_traj_pushed_up = TA_traj[j]+360
                TA_to_append.append(TA_traj_pushed_up)
            else:
                TA_to_append.append(TA_traj[j])
        dict_ridge[key_to_change].append(TA_to_append)    

    return dict_ridge

In [ ]:
def excludeTATracesNON_0_360(dict_ridge):
    #Exclude from dict all trials where the TA traj is outside 0 to 360 
    values_list = list(dict_ridge.values())
    key_list = list(dict_ridge.keys()) #trial
    for i in np.arange(len(key_list)):
        TA_traj = values_list[i][4]
        for j in np.arange(len(TA_traj)):
            #print(len(TA_traj))
            if np.nanmean(TA_traj[j])>350:
                TA_traj[j] = []
            elif np.nanmean(TA_traj[j]) < 10:
                TA_traj[j] = []
    return dict_ridge   

In [ ]:
def excludeTATracesHighDerivative(dict_ridge):
    #Exclude from dict all trials where the TA traj derivative is high
    values_list = list(dict_ridge.values())
    key_list = list(dict_ridge.keys()) #trial
    for i in np.arange(len(key_list)):
        TA_traj = values_list[i][4]
        for j in np.arange(len(TA_traj)): # step
            TA_diff = np.diff(TA_traj[j])
            if np.any(TA_diff>8) or np.any(TA_diff<-8):
                TA_traj[j] = []
    return dict_ridge   

In [ ]:
def transpose_TA_ipsi_contra_within0_360_range(TA_traj):
    if np.nanmean(TA_traj)>400:
        TA_traj = TA_traj-360
    elif np.nanmean(TA_traj)<-40:
        TA_traj = TA_traj+360
    return TA_traj

In [ ]:
def decide_TA_traj_wrt_IpsiorContraStep(TA_traj_listwrtR, TA_traj_listwrtL):
    TA_wrtIpsiStep = []
    TA_wrtContraStep = []
    if np.nanmean(TA_traj_listwrtR[110:130]) < 180:
        TA_wrtIpsiStep = TA_traj_listwrtR
    if np.nanmean(TA_traj_listwrtL[110:130]) > 180:
        TA_wrtIpsiStep = -TA_traj_listwrtL+360
    if np.nanmean(TA_traj_listwrtR[110:130]) > 180:
        TA_wrtContraStep = -TA_traj_listwrtR+360
    if np.nanmean(TA_traj_listwrtL[110:130]) < 180:
        TA_wrtContraStep = TA_traj_listwrtL
    return TA_wrtIpsiStep, TA_wrtContraStep

In [ ]:
def decide_HA_traj_wrt_IpsiorContraStep(TA_traj_listwrtR, TA_traj_listwrtL, Hip_traj_list_wrt_R, Hip_traj_list_wrt_L):
    Hip_wrtIpsiStep = []
    Hip_wrtContraStep = []
    if np.nanmean(TA_traj_listwrtR[110:130]) < 180:
        Hip_wrtIpsiStep = Hip_traj_list_wrt_R
    if np.nanmean(TA_traj_listwrtL[110:130]) > 180:
        Hip_wrtIpsiStep = -Hip_traj_list_wrt_L+360+180
    if np.nanmean(TA_traj_listwrtR[110:130]) > 180:
        Hip_wrtContraStep = -Hip_traj_list_wrt_R+360+180
    if np.nanmean(TA_traj_listwrtL[110:130]) < 180:
        Hip_wrtContraStep = Hip_traj_list_wrt_L
    return Hip_wrtIpsiStep, Hip_wrtContraStep

In [ ]:
def excludeemptyarray(dict_ridge):
    #Exclude from dict all trials where the TA traj derivative is high and return to COL 15
    values_list = list(dict_ridge.values())
    key_list = list(dict_ridge.keys()) #trial
    for i in np.arange(len(key_list)):
        TA_traj_listwrtR = values_list[i][4]
        TA_traj_listwrtL = values_list[i][6]
        COM_traj_list = values_list[i][9]
        COMX_traj_list = values_list[i][11]
        #COMdist_traj_list = values_list[i][12]
        Hip_traj_list_wrt_R = values_list[i][12]
        Hip_traj_list_wrt_L = values_list[i][17]
        StepAnlge_traj_list = values_list[i][13]
        ContraStep_traj_list = values_list[i][14]
        Step_x_traj_list = values_list[i][3]
        values_to_append = []
        for j, k in zip(np.arange(len(TA_traj_listwrtR)), np.arange(len(TA_traj_listwrtL))):
            if len(TA_traj_listwrtR[j]) and len(COM_traj_list[j]):
                #Transpose traces beyond 0-360 and exclude traces that are still beyon range
                TA_traj_listwrtL_T = transpose_TA_ipsi_contra_within0_360_range(TA_traj_listwrtL[k])
                TA_traj_listwrtR_T = transpose_TA_ipsi_contra_within0_360_range(TA_traj_listwrtR[j])
                #divide TA traces based on contra step
                TA_wrtIpsiStep, TA_wrtContraStep = decide_TA_traj_wrt_IpsiorContraStep(TA_traj_listwrtR_T, \
                                                                                       TA_traj_listwrtL_T)
                HA_wrtIpsiStep, HA_wrtContraStep = decide_HA_traj_wrt_IpsiorContraStep(TA_traj_listwrtR_T, \
                                                                                       TA_traj_listwrtL_T, \
                                                                                       Hip_traj_list_wrt_R[j], \
                                                                                       Hip_traj_list_wrt_L[k])
                #Assign to value in dict
                values_to_append.append([TA_traj_listwrtR[j], COM_traj_list[j], COMX_traj_list[j], \
                                         #COMdist_traj_list[j], Hip_traj_list[j], StepAnlge_traj_list[j],\
                                         Hip_traj_list_wrt_R[j], StepAnlge_traj_list[j],\
                                         Step_x_traj_list[j], StepAnlge_traj_list[j], TA_wrtContraStep, \
                                         TA_wrtIpsiStep, TA_traj_listwrtL[k], ContraStep_traj_list[j], \
                                         Hip_traj_list_wrt_L[k], HA_wrtIpsiStep, HA_wrtContraStep])

        dict_ridge[key_list[i]].append(values_to_append)
    return dict_ridge 

In [ ]:
# def excludeemptyarray_wrtLstep(dict_ridge):
#     #Exclude from dict all trials where the TA traj derivative is high and return to COL 15
#     values_list = list(dict_ridge.values())
#     key_list = list(dict_ridge.keys()) #trial
#     for i in np.arange(len(key_list)):
#         TA_traj_listwrtL = values_list[i][6]
#         COM_traj_list = values_list[i][9]
#         COMX_traj_list = values_list[i][11]
#         #COMdist_traj_list = values_list[i][12]
#         Hip_traj_list = values_list[i][12]
#         StepAnlge_traj_list = values_list[i][13]
#         ContraStep_traj_list = values_list[i][14]
#         Step_x_traj_list = values_list[i][3]
#         values_to_append = []
#         #print(len(TA_traj_listwrtL))
#         for j in np.arange(len(TA_traj_listwrtL)):
#             if len(TA_traj_listwrtL[j]):# and len(COM_traj_list[j]):
#                 values_to_append.append([TA_traj_listwrtL[j]])#, COM_traj_list[j], COMX_traj_list[j], \
#                                          #COMdist_traj_list[j], Hip_traj_list[j], StepAnlge_traj_list[j],\
# #                                         Hip_traj_list[j], StepAnlge_traj_list[j],\
# #                                         Step_x_traj_list[j], ContraStep_traj_list[j]])

#         dict_ridge[key_list[i]].append(values_to_append)
#     return dict_ridge 

In [ ]:
def firstNonNan(listfloats):
    i = 0
    for item in listfloats:
        i += 1
        if math.isnan(item) == False:
            return i

In [ ]:
#write function to compute speed of COM
def return_COM_speed(dict_ridge):
    values_list = list(dict_ridge.values())
    key_list = list(dict_ridge.keys()) #trial
    for i in np.arange(len(key_list)):
        #print(values_list[14][-1])

        COM_traj_list = values_list[i][-1]# last value in dict with COM empty array excluded
        COM_to_append = []
        for j in np.arange(len(COM_traj_list)):
            COM_traj = COM_traj_list[j][1]
            Centroid_DIST_traj_subtract1stelement = COM_traj-COM_traj[0]
            Centr_vel = (Centroid_DIST_traj_subtract1stelement[-1])/np.size(COM_traj)
            Centr_vel_pixelPersecond = Centr_vel *300 #300 Hz frames
            COM_to_append.append(Centr_vel_pixelPersecond)
        dict_ridge[key_list[i]].append(COM_to_append)  
    return dict_ridge   

In [ ]:
#write function to compute speed of COM
def return_TA_integral_Rstep(dict_ridge):
    values_list = list(dict_ridge.values())
    key_list = list(dict_ridge.keys()) #trial
    for i in np.arange(len(key_list)):
        TA_traj_list = values_list[i][-2]# last value in dict with TA empty array excluded
        #print(TA_traj_list)
        TA_to_append = []
        for j in np.arange(len(TA_traj_list)):
            TA_traj = TA_traj_list[j][0]
            TA_traj_diff = np.diff(TA_traj)
            TA_traj_integral = np.trapz(TA_traj_diff[120:150])
            TA_to_append.append(TA_traj_integral)
        dict_ridge[key_list[i]].append(TA_to_append)  
    return dict_ridge  

In [ ]:
#Write script to pre-process and organize all pert trial into python dict

search_key = ['4mm', '5mm', '8mm', '10mm']

dict_preprocessed_all = defaultdict(dict)

for i in np.arange(len(search_key)):
    dict_ridge = dict_ridge_all[search_key[i]]
    dict_ridge_ridge_pos = assign_dict_value_ridge_pos(dict_ridge)
    dict_ridge_el_pert_trial = eliminate_pert_trials_from_dict(dict_ridge_ridge_pos)
    dict_ridge_excl_nan_traces = excludeNaNTATraces(dict_ridge_el_pert_trial)
    dict_TA_transpose = transpose_traces_0_360_range(dict_ridge_excl_nan_traces)
    dict_TA_transpose_btw_0_360_der_excluded_without_empty_array = excludeemptyarray(dict_TA_transpose)#dict_TA_transpose_btw_0_360)
    dict_preprocessed_all[search_key[i]] = dict_TA_transpose_btw_0_360_der_excluded_without_empty_array

In [ ]:
# #Write script to pre-process and organize all pert trial into python dict wrt L step

# search_key = ['4mm', '5mm', '8mm', '10mm']

# dict_preprocessed_all_Lstep = defaultdict(dict)

# for i in np.arange(len(search_key)):
#     dict_ridge = dict_ridge_all[search_key[i]]
#     dict_ridge_ridge_pos = assign_dict_value_ridge_pos(dict_ridge)
#     dict_ridge_el_pert_trial = eliminate_pert_trials_from_dict(dict_ridge_ridge_pos)
#     dict_ridge_excl_nan_traces = excludeNaNTATraces(dict_ridge_el_pert_trial)
#     dict_TA_transposeL = transpose_traces_0_360_range(dict_ridge_excl_nan_traces)
#     dict_TA_transpose_btw_0_360_der_excluded_without_empty_arrayL = excludeemptyarray_wrtLstep(dict_TA_transposeL)#dict_TA_transpose_btw_0_360)
#     dict_preprocessed_all_Lstep[search_key[i]] = dict_TA_transpose_btw_0_360_der_excluded_without_empty_arrayL

In [ ]:
bla = dict_ridge_all['4mm']

values_list = list(bla.values())
key_list = list(bla.keys())

for i in np.arange(len(key_list)-1):
    TA_traj = values_list[i][3]
    step_L = values_list[i][5]
    #fig, ax1 = plt.subplots(1, 1, figsize=(25,15))
    for j, k in zip(np.arange(len(TA_traj)), np.arange(len(step_L))):
         plt.plot(TA_traj[j])
         plt.plot(step_L[k])
         plt.xlim(110, 150)

In [ ]:
# #Write script to pre-process and organize all pert trial into python dict

# search_key = ['4mm', '5mm', '8mm', '10mm']
# dict_preprocessed_all = defaultdict(dict)

# for i in np.arange(len(search_key)):
#     dict_ridge = dict_ridge_all[search_key[i]]
#     dict_ridge = assign_dict_value_ridge_pos(dict_ridge)
#     dict_ridge = eliminate_pert_trials_from_dict(dict_ridge)
#     dict_ridge = excludeNaNTATraces(dict_ridge)
#     dict_TA_transpose = transpose_traces_0_360_range(dict_ridge)
#     print(dict_TA_transpose.keys())
#     dict_TA_transpose_btw_0_360 = excludeTATracesNON_0_360(dict_TA_transpose)    
#     dict_preprocessed_all[search_key[i]] = dict_TA_transpose_btw_0_360

In [ ]:
bla = dict_ridge_all['4mm']

values_list = list(bla.values())
key_list = list(bla.keys())

for i in np.arange(len(key_list)-1):
    TA_traj = values_list[i][11]
    #print(TA_traj)

In [ ]:
#PLOTS BEFORE PRE-PROCESS wrt R STEP
#Plot trials for a specific width where all trials from the same animal are plot in the same plot (tot 15 plots) 
MouseID_key = ['M48', 'M49', 'M50', 'M51', 'M52', 'M53', 'M54', 'M55', 'M56', 'M57', 'M58', 'M59', 'M60', 'M61', 'M62']

for i in np.arange(len(MouseID_key)):
    dict_ridge_Xwidth = dict_ridge_all['4mm']
    dict_ridge_XwidthXmouseID = dict(filter(lambda item: MouseID_key[i] in item[0], dict_ridge_Xwidth.items())) 
    values_list = list(dict_ridge_XwidthXmouseID.values())
    key_list = list(dict_ridge_XwidthXmouseID.keys()) 
#    fig, ax1 = plt.subplots(1, 1, figsize=(15,6))
    fig = plt.figure()
    ax = fig.add_subplot(111)    
    for i in np.arange(len(key_list)):
        TA_traj_wrt_step = values_list[i][4]
        #print(TA_traj_wrt_step)
        for j in np.arange(len(TA_traj_wrt_step)):
            plt.plot(TA_traj_wrt_step[j] ,'y')
            plt.title("{i}".format(i = key_list[i]))
            #plt.ylim(200, 300)


In [ ]:
#PLOTS BEFORE PRE-PROCESS wrt L STEP
#Plot trials for a specific width where all trials from the same animal are plot in the same plot (tot 15 plots) 
MouseID_key = ['M48', 'M49', 'M50', 'M51', 'M52', 'M53', 'M54', 'M55', 'M56', 'M57', 'M58', 'M59', 'M60', 'M61', 'M62']

for i in np.arange(len(MouseID_key)):
    dict_ridge_Xwidth = dict_ridge_all['4mm']
    dict_ridge_XwidthXmouseID = dict(filter(lambda item: MouseID_key[i] in item[0], dict_ridge_Xwidth.items())) 
    values_list = list(dict_ridge_XwidthXmouseID.values())
    key_list = list(dict_ridge_XwidthXmouseID.keys()) 
#    fig, ax1 = plt.subplots(1, 1, figsize=(15,6))
    fig = plt.figure()
    ax = fig.add_subplot(111)    
    for i in np.arange(len(key_list)):
        TA_traj_wrt_step = values_list[i][6]
        #print(TA_traj_wrt_step)
        for j in np.arange(len(TA_traj_wrt_step)):
            plt.plot(TA_traj_wrt_step[j] ,'y')
            plt.title("{i}".format(i = key_list[i]))
            #plt.ylim(200, 300)

In [ ]:
def excludeTA_outrange(TA):
    if any(i < -30 or i > 250 for i in TA):
        TA = []
    return TA

In [ ]:
#### PLOT POST PROCESS
#Plot post-process TA wrt CONTRA step diff color for diff Mice
Widths_key = ['4mm']#, '5mm', '8mm', '10mm']
MouseID_key = ['M48']#, 'M49', 'M50', 'M51', 'M52', 'M53', 'M54', 'M55', 'M56', 'M57', 'M58', 'M59', 'M60', 'M61', 'M62']
foo_step_list = []
TA_up_thresh = []
TA_below_thresh = []
clrs = sns.color_palette("husl", len(MouseID_key))
dict_traces_perMouseID_means = defaultdict(dict)
dict_TA_125_contra = defaultdict(dict)

for l in np.arange(len(Widths_key)):
    fig, ax1 = plt.subplots(1, 1, figsize=(15,10))
    list_TA_125_contra = []
    for i in np.arange(len(MouseID_key)):
        dict_ridge_Xwidth = dict_preprocessed_all[Widths_key[l]]
        dict_ridge_XwidthXmouseID = dict(filter(lambda item: MouseID_key[i] in item[0], dict_ridge_Xwidth.items())) 
        values_list = list(dict_ridge_XwidthXmouseID.values())
        key_list = list(dict_ridge_XwidthXmouseID.keys()) 
        #Initialize
        class_value_list_contra = []
        class_value_list_ipsi = []
        for k in np.arange(len(key_list)):
            TA_traj_wrt_step = values_list[k][-1]
            COMX_traj_wrt_step = values_list[k][-1]    
            for j in np.arange(len(TA_traj_wrt_step)):
                #Compute only TA traces that are within limit
                TA_ipsi_inrange = excludeTA_outrange(COMX_traj_wrt_step[j][8])
                TA_contra_inrange = excludeTA_outrange(COMX_traj_wrt_step[j][7])
                if len(TA_contra_inrange):
                    fig, ax1 = plt.subplots(1, 1, figsize=(15,10))
                    #plt.plot(TA_ipsi_inrange ,color = 'b', lw=0.4, alpha = 0.5)
                    plt.plot(TA_contra_inrange ,color = 'r', lw=0.4, alpha = 0.5)
                    TA_binning_value = TA_contra_inrange[100]
                    TA_binning_value_list.append(TA_binning_value)
                    plt.title("{i}".format(i = key_list[k]))
                    TA_125_contra = TA_contra_inrange[125]
                    list_TA_125_contra.append(TA_125_contra)
                    if len(TA_ipsi_inrange) == 250:
                        class_value_list_ipsi.append(TA_ipsi_inrange)
                    if len(TA_contra_inrange) == 250:
                        class_value_list_contra.append(TA_contra_inrange)
            dict_TA_125_contra[Widths_key[l]] = list_TA_125_contra
        list_ipsi_mean = np.nanmean(class_value_list_ipsi, axis=0)
        list_contra_mean = np.nanmean(class_value_list_contra, axis=0)
        dict_traces_perMouseID[Widths_key[l]][MouseID_key[i]] = [list_ipsi_mean, \
                                                                 list_contra_mean]                


In [ ]:
#### PLOT POST PROCESS
#Plot post-process TA wrt CONTRA step diff color for diff Mice
Widths_key = ['4mm', '5mm', '8mm', '10mm']
MouseID_key = ['M48', 'M49', 'M50', 'M51', 'M52', 'M53', 'M54', 'M55', 'M56', 'M57', 'M58', 'M59', 'M60', 'M61', 'M62']
foo_step_list = []
TA_up_thresh = []
TA_below_thresh = []
clrs = sns.color_palette("husl", len(MouseID_key))
dict_traces_perMouseID_means = defaultdict(dict)
dict_TA_125_contra = defaultdict(dict)

for l in np.arange(len(Widths_key)):
    fig, ax1 = plt.subplots(1, 1, figsize=(15,10))
    list_TA_125_contra_and_COMY_vel = []
    for i in np.arange(len(MouseID_key)):
        dict_ridge_Xwidth = dict_preprocessed_all[Widths_key[l]]
        dict_ridge_XwidthXmouseID = dict(filter(lambda item: MouseID_key[i] in item[0], dict_ridge_Xwidth.items())) 
        values_list = list(dict_ridge_XwidthXmouseID.values())
        key_list = list(dict_ridge_XwidthXmouseID.keys()) 
        #Initialize
        class_value_list_contra = []
        class_value_list_ipsi = []
        for k in np.arange(len(key_list)):
            TA_traj_wrt_step = values_list[k][-1]
            COMX_traj_wrt_step = values_list[k][-1]    
            for j in np.arange(len(TA_traj_wrt_step)):
                #Compute only TA traces that are within limit
                TA_ipsi_inrange = excludeTA_outrange(COMX_traj_wrt_step[j][8])
                TA_contra_inrange = excludeTA_outrange(COMX_traj_wrt_step[j][7])
                if len(COMX_traj_wrt_step[j][1])==250 and len(TA_contra_inrange):
                    #plt.plot(TA_ipsi_inrange ,color = clrs[i], lw=0.8, alpha = 0.5)
                    plt.plot(COMX_traj_wrt_step[j][1]-COMX_traj_wrt_step[j][1][0] ,color = clrs[i], lw=0.8, alpha = 0.5)
                    #TA_binning_value = COMX_traj_wrt_step[j][1][250]-COMX_traj_wrt_step[j][1][0]
                    #TA_binning_value_list.append(TA_binning_value)
                    plt.title("{i}".format(i = key_list[k]))
                    #print(COMX_traj_wrt_step[j][1])
                    COMY_vel = COMX_traj_wrt_step[j][1][249]-COMX_traj_wrt_step[j][1][0]
                    list_TA_125_contra_and_COMY_vel.append([COMY_vel, (TA_contra_inrange[130]-TA_contra_inrange[100])])
                    #list_COMY_vel.append(COMY_vel)
                    if len(TA_ipsi_inrange) == 250:
                        class_value_list_ipsi.append(TA_ipsi_inrange)
                    if len(TA_contra_inrange) == 250:
                        class_value_list_contra.append(TA_contra_inrange)
            dict_TA_125_contra[Widths_key[l]] = [list_TA_125_contra_and_COMY_vel]
        list_ipsi_mean = np.nanmean(class_value_list_ipsi, axis=0)
        list_contra_mean = np.nanmean(class_value_list_contra, axis=0)
        dict_traces_perMouseID[Widths_key[l]][MouseID_key[i]] = [list_ipsi_mean, \
                                                                 list_contra_mean] 

In [ ]:
dict_TA_125_contra['4mm'][0][0]

In [ ]:
#Scatterplot to check that TA at beg of R step determines movement after R step peak velocity
Widths_key = ['10mm']#, '5mm', '8mm', '10mm']

for i in np.arange(len(Widths_key)):
    dict_list = dict_TA_125_contra[Widths_key[i]][0]
    #values_list = list(dict_list.values())
    scatter_1 = [el [0] for el in dict_list]
    scatter_2 = [el [1] for el in dict_list]
    #print(scatter_1)
    plt.scatter(scatter_1, scatter_2)
    plt.ylim(-80,110)

In [ ]:
#Plot means TA traces per mouse ID
values_list = list(dict_traces_perMouseID.values())
key_list = list(dict_traces_perMouseID.keys()) 
clrs = sns.color_palette("husl", 4)
fig, ax1 = plt.subplots(1, 1, figsize=(15,10))
TA_all_traces_one_width_ipsi = []
TA_all_traces_one_width_contra = []
for l in np.arange(len(key_list)):
    TA_per_mouse = values_list[l]
    values_list_per_mouse = list(TA_per_mouse.values())
    key_list_per_mouse = list(TA_per_mouse.keys()) 
    for color_i, i in zip(color_idx, np.arange(len(key_list_per_mouse))):
#         #Plot here
        plt.plot(values_list_per_mouse[i][0], c=clrs[l], lw=0.5, alpha = 0.7)
        #plt.plot(values_list_per_mouse[i][1], c=clrs[l], lw=0.5, alpha = 0.7)
        #plt.title('TA traces wrt Contra step')
        TA_all_traces_one_width_ipsi.append(values_list_per_mouse[i][0])
        #TA_all_traces_one_width_contra.append(values_list_per_mouse[i][1])

    TA_mean_one_width_ipsi = np.nanmean(TA_all_traces_one_width_ipsi, axis=0)
    #TA_mean_one_width_contra = np.nanmean(TA_all_traces_one_width_contra, axis=0)
    plt.plot(TA_mean_one_width_ipsi, c=clrs[l], lw=3, alpha = 0.7, label = Widths_key[l])
    #plt.plot(TA_mean_one_width_contra, '--',c=clrs[l], lw=3, alpha = 0.7, label = Widths_key[l])
    plt.legend(loc="lower left")
    plt.axvline(125,0,360, color = 'red')
plt.savefig('out3')
plt.show()

In [ ]:
def return_Mean_STD_forPSTH(array_value_dict):
    mean_array = np.nanmean(array_value_dict, axis = 0)
    STD_array = stats.sem(array_value_dict, nan_policy='omit')
    return mean_array, STD_array

In [ ]:
#### PLOT POST PROCESS
#Plot post-process TA wrt R step diff color for diff widths
search_key = ['4mm', '5mm', '8mm', '10mm']
fig, ax1 = plt.subplots(1, 1, figsize=(15,10))
clrs = sns.color_palette("husl", len(MouseID_key))
mean_big_list = []
std_big_list = []
mean_small_list = []
std_small_list = []
mean_combo_list = []

a = 0
for k in np.arange(len(search_key)):
    TA_up_thresh = []
    TA_below_thresh = []
    dict_ridge_Xwidth = dict_preprocessed_all[search_key[k]]
    values_list = list(dict_ridge_Xwidth.values())
    key_list = list(dict_ridge_Xwidth.keys()) 
    for i in np.arange(len(key_list)):
        TA_traj_wrt_step = values_list[i][-1]
        COMX_traj_wrt_step = values_list[i][-1] 
        for j in np.arange(len(TA_traj_wrt_step)):
            Tail_idx = 7
            Hip_idx = 13
            if len(COMX_traj_wrt_step[j][Tail_idx]):# Plot Tail Angle trajectory
                a = a+1
                #print(a)
                plt.plot(COMX_traj_wrt_step[j][Tail_idx] ,color = 'b', lw=0.5, alpha = 0.1)
                plt.ylim(-50,420)
                if len(COMX_traj_wrt_step[j][Tail_idx]) == 250:
                    TA_below_thresh.append((np.diff(COMX_traj_wrt_step[j][Tail_idx]))*0.5*(1/3)*49)#Ang Mom TA
            if len(COMX_traj_wrt_step[j][Hip_idx]): #Plot Hip Angle trajectory
                plt.plot(COMX_traj_wrt_step[j][Hip_idx] ,color = 'r', lw=0.5, alpha = 0.1)
                if len(COMX_traj_wrt_step[j][Hip_idx]) == 250:
                    TA_up_thresh.append((np.diff(COMX_traj_wrt_step[j][Hip_idx]))*20*0.5*4)#Ang Mom HA
    mean_big, std_big = return_Mean_STD_forPSTH(TA_up_thresh)
    mean_small, std_small = return_Mean_STD_forPSTH(TA_below_thresh)
    mean_big_list.append(mean_big)
    std_big_list.append(std_big)
    mean_small_list.append(mean_small)
    std_small_list.append(std_small)
    mean_combo_list.append(mean_small+mean_big)


In [ ]:
print(len())

In [ ]:
def plot_PSTH_Mean_STD_label_color_pre_assigned(mean_array, std_array, ax, ls_param, label_strings):
    clrs = sns.color_palette("husl", len(mean_array))
    #label_strings = ['4mm', '5mm', '8mm', '10mm']#, 'Ipsi 4mm', 'Contra 4mm', 'Ipsi 4mm', 'Contra 4mm']
#    _, ax = plt.subplots(figsize=(12, 8))

    with sns.axes_style("darkgrid"):
        for i in range(len(mean_array)):
            epochs = list(range(len(mean_array[i])))
            ax.plot(epochs, mean_array[i], c=clrs[i], label = label_strings[i], ls = ls_param)
            ax.fill_between(epochs, mean_array[i]-std_array[i], mean_array[i]+std_array[i], \
                            alpha=0.3, facecolor=clrs[i])
            ax.legend(loc="lower left")
            ax.set_xlabel('Frame number (FR = 300 Hz)')
            ax.set_ylabel('Angular Momentum (g-cm2/s)')
            ax.axvline(125,0,360, color = 'red')
            #ax.axvline(140,0,360, color = 'red')

    return ax


In [ ]:
#IPSI
_, ax = plt.subplots(figsize=(12, 8))
clrs = sns.color_palette("husl", 8)
label_strings_bl = ['4mm Hip', '5mm Hip', '8mm Hip', '10mm Hip']
label_strings_sl = ['4mm Tail', '5mm Tail', '8mm Tail', '10mm Tail']
ls_param_bl = '--'
ls_param_sl = '-'
ls_param_combo = 'dotted'
plot_PSTH_Mean_STD_label_color_pre_assigned(mean_big_list, std_big_list, ax, ls_param_bl, label_strings_bl) 
plot_PSTH_Mean_STD_label_color_pre_assigned(mean_small_list, std_small_list, ax, ls_param_sl, label_strings_sl)  
# plot_PSTH_Mean_STD_label_color_pre_assigned(mean_combo_list, \
#                                             std_small_list, ax, ls_param_combo, label_strings_sl)    

plt.xlim(50, 200)
plt.savefig('out')


In [ ]:
#Contra
_, ax = plt.subplots(figsize=(12, 8))
clrs = sns.color_palette("husl", 8)
label_strings_bl = ['4mm Hip', '5mm Hip', '8mm Hip', '10mm Hip']
label_strings_sl = ['4mm Tail', '5mm Tail', '8mm Tail', '10mm Tail']
ls_param_bl = '--'
ls_param_sl = '-'
ls_param_combo = 'dotted'
plot_PSTH_Mean_STD_label_color_pre_assigned(mean_big_list, std_big_list, ax, ls_param_bl, label_strings_bl) 
plot_PSTH_Mean_STD_label_color_pre_assigned(mean_small_list, std_small_list, ax, ls_param_sl, label_strings_sl)  
# plot_PSTH_Mean_STD_label_color_pre_assigned(mean_combo_list, \
#                                             std_small_list, ax, ls_param_combo, label_strings_sl)    

plt.xlim(50, 200)
plt.savefig('out')

In [ ]:
#wrt R step
_, ax = plt.subplots(figsize=(12, 8))
clrs = sns.color_palette("husl", 8)
label_strings_bl = ['4mm Hip', '5mm Hip', '8mm Hip', '10mm Hip']
label_strings_sl = ['4mm Tail', '5mm Tail', '8mm Tail', '10mm Tail']
ls_param_bl = '--'
ls_param_sl = '-'
plot_PSTH_Mean_STD_label_color_pre_assigned(mean_big_list, std_big_list, ax, ls_param_bl, label_strings_bl) 
plot_PSTH_Mean_STD_label_color_pre_assigned(mean_small_list, std_small_list, ax, ls_param_sl, label_strings_sl)    
plt.xlim(70, 170)
plt.savefig('out')

In [ ]:
#wrt L step
_, ax = plt.subplots(figsize=(12, 8))
clrs = sns.color_palette("husl", 8)
label_strings_bl = ['4mm Hip', '5mm Hip', '8mm Hip', '10mm Hip']
label_strings_sl = ['4mm Tail', '5mm Tail', '8mm Tail', '10mm Tail']
ls_param_bl = '--'
ls_param_sl = '-'
plot_PSTH_Mean_STD_label_color_pre_assigned(mean_big_list, std_big_list, ax, ls_param_bl, label_strings_bl) 
plot_PSTH_Mean_STD_label_color_pre_assigned(mean_small_list, std_small_list, ax, ls_param_sl, label_strings_sl)    
plt.xlim(70, 170)
plt.savefig('out')

In [ ]:
def assign_histogram_classifier(lag, max_height, thresh = 0):
    classifier = []
    if lag > thresh and max_height > 0:
        classifier = 0
    elif lag > thresh and max_height < 0:
        classifier = 1
    elif lag < thresh and max_height > 0:
        classifier = 2
    elif lag < thresh and max_height < 0:
        classifier = 3
    return classifier

In [ ]:
def assign_quadrant_classifier(TA, no_quad = 6):
    lst = np.arange(361)
    chunks_list = np.array_split(lst, no_quad)
    classifier = []
    for i in np.arange(len(chunks_list)):
        first_value = chunks_list[i][0]
        last_value = chunks_list[i][-1]
        if first_value <= TA < last_value:
            classifier = i
    return classifier

In [ ]:
def splitsequenceequally(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [ ]:
def assign_quadrant_classifier_equal_number(TA, no_quad, list_of_TAMeans_equally_split):
    classifier = []
    #print(len(list_of_TAMeans_equally_split))
    for i in np.arange(len(list_of_TAMeans_equally_split)):
        first_value = list_of_TAMeans_equally_split[i][0]
        last_value = list_of_TAMeans_equally_split[i][-1]
        if first_value <= TA <= last_value:
            #print(first_value, last_value, i)
            classifier = i
    return classifier

In [ ]:
#Split traces into N equally populated arrays
MouseID_key = ['M48', 'M49', 'M50', 'M51', 'M52', 'M53', 'M54', 'M55', 'M56', 'M57', 'M58', 'M59', 'M60', 'M61', 'M62']

dict_ridge_4mm = dict_preprocessed_all['all']
dict_traces_divided_on_TA_criteria = defaultdict(dict)
no_quad_to_plot = 100
TA_classifier_criteria_array = np.array(TA_binning_value_list)
#print(TA_classifier_criteria_array)
TA_classifier_criteria_array_woutNaN = TA_classifier_criteria_array[~numpy.isnan(TA_classifier_criteria_array)]
TA_classifier_criteria_array_woutNaN.sort()
list_of_TAtraces_eq_split = splitsequenceequally(TA_classifier_criteria_array_woutNaN,no_quad_to_plot)
for i in np.arange(len(MouseID_key)):
    dict_ridge_Xwidth = dict_preprocessed_all['4mm']
    dict_ridge_XwidthXmouseID = dict(filter(lambda item: MouseID_key[i] in item[0], dict_ridge_Xwidth.items())) 
    values_list = list(dict_ridge_XwidthXmouseID.values())
    key_list = list(dict_ridge_XwidthXmouseID.keys())
    class_value_list = []
    for k in np.arange(len(key_list)):
        TA_traj_wrt_step = values_list[k][-1]    
        for j in np.arange(len(TA_traj_wrt_step)):
            if len(TA_traj_wrt_step[j][7]) == 250:
                Trial_classvalue = values_list[j][7]
                #print(TA_traj_wrt_step, j)
                TA_traj = TA_traj_wrt_step[j][7] ######### CHANGE THIS LINE TO ADJUST CLASSIFIER
                #print(list_of_TAtraces_eq_split)
                TA_classifier_criteria = TA_traj[100]
                quadrant_value = assign_quadrant_classifier_equal_number(TA_classifier_criteria, no_quad_to_plot, list_of_TAtraces_eq_split)
                #print(TA_classifier_criteria, quadrant_value)
                class_value_list.append([TA_traj, quadrant_value])

    dict_traces_divided_on_TA_criteria[MouseID_key[i]] = class_value_list

In [ ]:
##### Assign TA list for each quadrant in dict
def compute_mean_values_insideDict(dict_traces_divided_on_TA_criteria, no_quad_to_plot):
    values_list = list(dict_traces_divided_on_TA_criteria.values())
    key_list = list(dict_traces_divided_on_TA_criteria.keys()) 
    dict_mean_quadrants = defaultdict(dict)
    print(len(key_list))
    for k in range(no_quad_to_plot):
        TA_traj_list = []
        for i in np.arange(len(key_list)):
            for j in np.arange(len(values_list[i])):
                TA_traj = values_list[i][j][0]
                quad_class = values_list[i][j][1]
                if quad_class == k and len(TA_traj)== 250:
                    TA_traj_list.append(TA_traj)
        dict_mean_quadrants[k] = TA_traj_list
    return dict_mean_quadrants

In [ ]:
#Plot quadrants
values_list = list(dict_mean_quadrants.values())
key_list = list(dict_mean_quadrants.keys()) 

for i in np.arange(len(key_list)):
    fig, ax1 = plt.subplots(1, 1, figsize=(15,6))
    values_quad = values_list[i]
    for j in np.arange(len(values_quad)):
        ax1.plot(values_quad[j])

In [ ]:
#Compute mean of values inside binned dictionary
dict_mean_quadrants = compute_mean_values_insideDict(dict_traces_divided_on_TA_criteria, no_quad_to_plot)

#Plot means
values_list = list(dict_mean_quadrants.values())
key_list = list(dict_mean_quadrants.keys()) 
color_idx = np.linspace(0, 1, no_quad_to_plot)
fig, ax1 = plt.subplots(1, 1, figsize=(15,10))

for color_i, i in zip(color_idx, np.arange(no_quad_to_plot)):#len(key_list)):
    mean_quadrant_traces = np.nanmean(dict_mean_quadrants[i], axis=0)
    data = mean_quadrant_traces#-np.nanmean(mean_quadrant_traces)
    #Plot here
    plt.plot(data, color=plt.cm.coolwarm(color_i), lw=1, alpha = 0.7, label="%s quadrant" % (i+1))
    plt.ylim(50,300)
plt.show()

In [ ]:
values_list = list(dict_mean_quadrants.values())
key_list = list(dict_mean_quadrants.keys()) 
color_idx = np.linspace(0, 1, no_quad_to_plot)
fig, ax1 = plt.subplots(1, 1, figsize=(15,10))

for color_i, i in zip(color_idx, np.arange(no_quad_to_plot)):#len(key_list)):
    mean_quadrant_traces = np.nanmean(dict_mean_quadrants[i], axis=0)
    data = mean_quadrant_traces#-np.nanmean(mean_quadrant_traces)
    #Plot here
    plt.plot(data, color=plt.cm.coolwarm(color_i), lw=1, alpha = 0.7, label="%s quadrant" % (i+1))
    plt.ylim(50,300)
plt.show()